In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

## Importation des données

In [2]:
df = pd.read_csv('/home/maylin/Downloads/Dataset_complete.csv')

In [25]:
capteurs = np.unique(df['target'])
for capt in capteurs:
  print('nombre de signaux reçus par le capeur',capt,':',len(df[df.target==capt]['seqnum']))
# le nombre de signaux reçus par les capteurs n'est pas le même
# certains capteurs n'ont pas reçu le signal émis par le client 

nombre de signaux reçus par le capeur 100 : 643
nombre de signaux reçus par le capeur 101 : 706
nombre de signaux reçus par le capeur 150 : 477
nombre de signaux reçus par le capeur 171 : 704
nombre de signaux reçus par le capeur 180 : 708
nombre de signaux reçus par le capeur 184 : 708


### Calcul de position 

Méthode analytique de calcul

In [5]:
def trackObject(x1,y1,r1,x2,y2,r2,x3,y3,r3):
  A = 2*x2 - 2*x1
  B = 2*y2 - 2*y1
  C = r1**2 - r2**2 - x1**2 + x2**2 - y1**2 + y2**2
  D = 2*x3 - 2*x2
  E = 2*y3 - 2*y2
  F = r2**2 - r3**2 - x2**2 + x3**2 - y2**2 + y3**2

  x = (C*E - F*B) / (E*A - B*D)
  y = (C*D - A*F) / (B*D - A*E)
  return x,y

In [6]:
import random

df2 = pd.DataFrame(columns=['seqnum1', 'x1', 'y1', 'r1', 'seqnum2', 'x2', 'y2', 'r2', 'seqnum3', 'x3', 'y3', 'r3', 'xTrue', 'yTrue', 'xCalc', 'yCalc'])

for i in range(len(df)):
    
    if (i%6 == 0):
        
        option1, option2, option3 = random.sample(range(i+1, i+7), 3)
        # vérifier que tous les seqnum sont présents dans le dataframe
        
        OK = False
        while OK == False:
            try : 
                r1 = float(df[df.seqnum == option1].range_skew)
                r2 = float(df[df.seqnum == option2].range_skew)
                r3 = float(df[df.seqnum == option3].range_skew)
                OK = True
            except:
                option1, option2, option3 = random.sample(range(i+1, i+7), 3)

        x1 = float(df[df.seqnum == option1].loc_target_x)
        y1 = float(df[df.seqnum == option1].loc_target_y)  
        x2 = float(df[df.seqnum == option2].loc_target_x)
        y2 = float(df[df.seqnum == option2].loc_target_y)  
        x3 = float(df[df.seqnum == option3].loc_target_x)
        y3 = float(df[df.seqnum == option3].loc_target_y) 
        xTrue = float(df[df.seqnum == option1].loc_initiator_x)
        yTrue = float(df[df.seqnum == option1].loc_initiator_y)
        
        xCalc, yCalc = trackObject(x1,y1,r1,x2,y2,r2,x3,y3,r3)
        
    df2 = df2.append({'x1':x1, 'y1':y1, 'r1':r1, 
                      'x2':x2, 'y2':y2, 'r2':r2, 
                      'x3':x3, 'y3':y3, 'r3':r3, 
                      'seqnum1': option1, 'seqnum2': option2, 'seqnum3': option3,
                      'xTrue':xTrue, 'yTrue':yTrue, 
                      'xCalc':xCalc, 'yCalc':yCalc} , ignore_index=True)

df2.drop_duplicates(keep = 'first', inplace=True)

/tmp/ipykernel_11009/3625771259.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'x1':x1, 'y1':y1, 'r1':r1,
/tmp/ipykernel_11009/3625771259.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'x1':x1, 'y1':y1, 'r1':r1,
/tmp/ipykernel_11009/3625771259.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'x1':x1, 'y1':y1, 'r1':r1,
/tmp/ipykernel_11009/3625771259.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'x1':x1, 'y1':y1, 'r1':r1,
/tmp/ipykernel_11009/3625771259.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

Calcul de la mse pour le calcul de la position

In [7]:
df2

,seqnum1,x1,y1,r1,seqnum2,x2,y2,r2,seqnum3,x3,y3,r3,xTrue,yTrue,xCalc,yCalc
0,6.0,-1.190,4.578,3.769005,4.0,-3.962,7.931,8.187508,5.0,-3.132,4.339,5.684735,2.120,2.793,2.712374,2.748514
6,11.0,-3.132,4.339,5.855792,8.0,6.169,7.657,6.316982,7.0,2.120,2.793,-0.023886,2.120,2.793,2.453657,2.530588
12,13.0,2.120,2.793,-0.308668,14.0,6.169,7.657,6.239798,18.0,-1.190,4.578,3.644861,2.120,2.793,2.067100,2.961737
18,24.0,-1.190,4.578,3.432981,20.0,6.169,7.657,6.225616,22.0,-3.962,7.931,7.911249,2.120,2.793,2.138457,2.576368
24,25.0,2.120,2.793,-0.113783,30.0,-1.190,4.578,3.647337,29.0,-3.132,4.339,5.946552,2.120,2.793,3.401453,5.407965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918,3922.0,-3.962,7.931,3.109838,3921.0,11.442,9.249,12.512439,3923.0,-3.132,4.339,5.025826,-1.034,8.476,-1.052828,8.881125
3924,3930.0,-1.190,4.578,3.756870,3928.0,-3.962,7.931,3.060272,3925.0,2.120,2.793,6.433924,-1.034,8.476,-1.335878,7.987880
3930,3933.0,11.442,9.249,13.033210,3932.0,6.169,7.657,7.227927,3935.0,-3.132,4.339,4.668393,-1.034,8.476,-9.802141,33.143498
3936,3939.0,11.442,9.249,12.380705,3940.0,-3.962,7.931,3.098760,3938.0,6.169,7.657,7.433257,-1.034,8.476,-1.078939,10.404241


Calcul distance

In [8]:
def score_distanceM(Xobserve:float, Yobserve:float, Xprevu:float, Yprevu:float) -> float:
    X=(Xobserve-Xprevu)**2
    Y=(Yobserve-Yprevu)**2
    return np.mean(np.sqrt(X+Y))

In [9]:
score_distanceM(Xobserve = df2.xTrue,
                Yobserve = df2.yTrue,
                Xprevu = df2.xCalc,
                Yprevu = df2.yCalc)

1.508295860572134